In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# Reading dataset
df = pd.read_csv("sales_data_sample.csv", encoding='ISO-8859-1')
pd.set_option('display.max_columns', None)

In [3]:
# Data cleaning
columns_to_drop = ['CONTACTFIRSTNAME', 'CONTACTLASTNAME', 'PHONE', 'ADDRESSLINE1', 
                   'ADDRESSLINE2', 'POSTALCODE', 'STATE', 'TERRITORY', 'CITY']
df.drop(columns=columns_to_drop, axis=1, inplace=True)

# Fixing wrong datatype
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])

# Creating product initial
df['PRODUCTINITIAL'] = df['PRODUCTCODE'].str[:3]
df.drop('PRODUCTCODE', axis=1, inplace=True)

In [ ]:
# Analysis questions

# 1. Monthly sales of each financial year
monthly_sales = df.groupby(['YEAR_ID', 'MONTH_ID'])['SALES'].sum().unstack(level=0)
fig = go.Figure()
for year in monthly_sales.columns:
    fig.add_trace(go.Scatter(
        x=monthly_sales.index,
        y=monthly_sales[year],
        mode='lines+markers',
        name=str(year),
        hovertemplate='Month: %{x}<br>Sales: %{y}<extra></extra>'
    ))
fig.update_layout(
    title='Monthly Sales',
    xaxis_title='Months',
    yaxis_title='Sales',
    xaxis=dict(tickmode='linear', tick0=1, dtick=1),
    hovermode='closest'
)
fig.show()

In [ ]:
# 2. Best year according to sales
yearly_sales = df.groupby(['YEAR_ID'])['SALES'].sum()
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=yearly_sales.index,
    y=yearly_sales.values,
    mode='lines+markers',
    name='Yearly sales',
    hovertemplate='Year: %{x}<br>Sales: %{y}<extra></extra>'
))
fig.update_layout(
    title='Yearly Sales',
    xaxis_title='Years',
    yaxis_title='Sales',
    xaxis=dict(tickmode='linear'),
    hovermode='closest'
)
fig.show()

In [ ]:
# 3. Best quarter for each product line
quarterly_sales = df.groupby(['PRODUCTLINE', 'QTR_ID'])['SALES'].sum().reset_index()
fig = px.bar(
    quarterly_sales,
    x='PRODUCTLINE',
    y='SALES',
    color='QTR_ID',
    title='Sales by Quarter for Each Product Line',
    labels={'SALES': 'Total Sales', 'QTR_ID': 'Quarter'},
    barmode='group'
)
fig.show()

In [ ]:
# 4. Product sold the most
product_sales = df.groupby('PRODUCTLINE')['SALES'].sum().reset_index()
fig = px.pie(
    product_sales,
    names='PRODUCTLINE',
    values='SALES',
    title='Sales Distribution by Product Line'
)
fig.show()

In [ ]:
# 5. Relationship between deal size and sales
fig = px.histogram(
    df,
    x='SALES',
    color='DEALSIZE',
    labels={'SALES': 'Sales', 'DEALSIZE': 'Deal Size'},
    title='Histogram of Sales by Deal Size'
)
fig.show()

In [ ]:
# 6. Top customer
customer_sales = df.groupby('CUSTOMERNAME')['SALES'].sum().reset_index()
top_10_customers = customer_sales.nlargest(10, 'SALES')
top_customer_name = top_10_customers.loc[top_10_customers['SALES'].idxmax(), 'CUSTOMERNAME']
top_customer_sales = top_10_customers['SALES'].max()

fig = px.scatter(
    top_10_customers,
    x='CUSTOMERNAME',
    y='SALES',
    title='Top 10 Sales by Customer',
    labels={'CUSTOMERNAME': 'Customer Name', 'SALES': 'Sales'},
    color_continuous_scale=px.colors.sequential.Plasma
)
fig.add_scatter(
    x=[top_customer_name],
    y=[top_customer_sales],
    mode='markers',
    marker=dict(color='red', size=12, symbol='star'),
    name='Top Customer'
)
fig.show()


In [ ]:
# 7. Country with best sales
country_sales = df.groupby('COUNTRY')['SALES'].sum().reset_index()
top_10_countries = country_sales.nlargest(10, 'SALES')
fig = px.bar(
    top_10_countries,
    x='COUNTRY',
    y='SALES',
    title='Top 10 Countries by Sales',
    labels={'COUNTRY': 'Country', 'SALES': 'Sales'},
    color='SALES',
    color_continuous_scale=px.colors.sequential.Plasma
)
fig.show()

fig = px.choropleth(
    top_10_countries,
    locations='COUNTRY',
    locationmode='country names',
    color='SALES',
    title='Top 10 Countries by Sales',
    labels={'SALES': 'Sales'},
    color_continuous_scale=px.colors.sequential.Plasma,
    projection='natural earth'
)
fig.show()

In [ ]:
# Basic data analysis
# Checking unique values
for column in df:
    print(f'Number of unique values in {column}:', df[column].nunique())

In [ ]:
# Data visualization
plt.rcParams['figure.figsize'] = [18, 16]
# Select only numeric columns for density plot
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns].plot(kind="density", subplots=True, layout=(4,4), sharex=False, sharey=False)
plt.show()


In [ ]:
# Checking for null values
print(df.isnull().sum())

# Sales status distribution
plt.rcParams['figure.figsize'] = [8, 5]
sns.countplot(y='STATUS', data=df, hue='YEAR_ID')
plt.title('Distribution of Sales Status')
plt.show()

# Sales by year
df.groupby(['YEAR_ID'])['SALES'].sum().plot(kind='bar', figsize=(6, 4))
plt.title('Sales by Year')
plt.show()


In [ ]:
# Comparing sales by quarter
quarter_sales = df.groupby(['YEAR_ID', 'QTR_ID']).agg({'SALES': 'sum'}).reset_index()
g = sns.catplot(
    y='SALES',
    x='QTR_ID',
    data=quarter_sales,
    kind="bar",
    hue='YEAR_ID',
    height=5,
    aspect=1.6
)
g.set_titles("Comparing Sales by Quarter")
plt.show()

In [ ]:
# RFM Analysis
rfm_columns = ['CUSTOMERNAME', 'ORDERNUMBER', 'ORDERDATE', 'SALES']
rfm_data = df[rfm_columns].copy()
rfm_data['ORDERDATE'] = pd.to_datetime(rfm_data['ORDERDATE'], errors='coerce')

# Setting reference date
reference_date = dt.datetime(2005, 5, 31)

# Creating RFM table
rfm_table = rfm_data.groupby('CUSTOMERNAME').agg({
    'ORDERDATE': lambda x: (reference_date - x.max()).days,
    'ORDERNUMBER': 'count',
    'SALES': 'sum'
}).rename(columns={
    'ORDERDATE': 'recency',
    'ORDERNUMBER': 'frequency',
    'SALES': 'monetary_value'
})

# RFM Scoring and K-means clustering
recency_quartiles = pd.qcut(rfm_table['recency'], 4, labels=list(range(3, -1, -1)))
frequency_quartiles = pd.qcut(rfm_table['frequency'], 4, labels=list(range(0, 4)))
monetary_quartiles = pd.qcut(rfm_table['monetary_value'], 4, labels=list(range(0, 4)))

rfm_scored = pd.DataFrame({
    'recency': recency_quartiles,
    'frequency': frequency_quartiles,
    'monetary_value': monetary_quartiles
})

rfm_array = rfm_scored.values

In [ ]:
# Finding optimal number of clusters
inertia_values = []
for i in range(1, 15):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(rfm_array)
    inertia_values.append([i, kmeans.inertia_])
cluster_data = pd.DataFrame(inertia_values, columns=['num_clusters', 'inertia'])

plt.figure(figsize=(10, 7))
sns.set(font_scale=1.4, style="whitegrid")
sns.lineplot(data=cluster_data, x="num_clusters", y="inertia").set(title="Elbow Method")
plt.show()


In [ ]:
# Visualization function
def plot_clusters():
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    grouped_data = rfm_table.groupby('cluster')
    for name, group in grouped_data:
        ax.scatter(
            group['recency'],
            group['monetary_value'],
            group['frequency'],
            s=50,
            alpha=0.6,
            edgecolors='w',
            label=name
        )
    ax.set_xlabel('Recency')
    ax.set_ylabel('Monetary Value')
    ax.set_zlabel('Frequency')
    plt.title('Visualization of Customer Segments')
    plt.legend()
    plt.show()


In [ ]:
# Model with 4 clusters
kmeans_model = KMeans(n_clusters=4, init='k-means++', max_iter=300)
clusters = kmeans_model.fit_predict(rfm_array)
rfm_scored['cluster'] = clusters
rfm_table['cluster'] = clusters

plot_clusters()

# Distribution of cluster sizes
print((rfm_table['cluster'].value_counts(normalize=True, sort=True) * 100).to_string())

# Statistics
print("Overall statistics:")
print(rfm_table.agg(['mean']))
print("\nCluster statistics:")
print(rfm_table.groupby('cluster').agg(['mean']))

# Plot cluster sizes
rfm_table['cluster'].value_counts().plot(kind='bar', figsize=(6, 4), title='Size of Customer Segments')
plt.show()


In [ ]:
# Assign segment names
def assign_segment(cluster):
    segment_map = {0: 'departing', 1: 'active', 2: 'inactive', 3: 'new'}
    return segment_map.get(cluster, 'new')

rfm_table['segment_name'] = rfm_table['cluster'].apply(assign_segment)

# Save results
rfm_table.to_csv('customer_segments.csv', index=False)
print(rfm_table.head())

In [ ]:
# -------------------------------------------
# RFM Analysis
# -------------------------------------------
today_date = df['ORDERDATE'].max() + pd.Timedelta(days=1)

rfm = df.groupby('CUSTOMERNAME').agg({
    'ORDERDATE': lambda x: (today_date - x.max()).days,
    'ORDERNUMBER': 'nunique',
    'SALES': 'sum'
}).reset_index()

rfm.columns = ['CustomerName', 'Recency', 'Frequency', 'Monetary']

# Assigning RFM scores
r_labels = [4, 3, 2, 1]
f_labels = [1, 2, 3, 4]
m_labels = [1, 2, 3, 4]

r_bins = pd.qcut(rfm['Recency'], q=4, labels=r_labels)
f_bins = pd.qcut(rfm['Frequency'].rank(method='first'), q=4, labels=f_labels)
m_bins = pd.qcut(rfm['Monetary'], q=4, labels=m_labels)

rfm = rfm.assign(R=r_bins.values, F=f_bins.values, M=m_bins.values)

# Combining RFM scores
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].astype(str).sum(axis=1)

# -------------------------------------------
# K-Means Clustering based on RFM Scores
# -------------------------------------------
rfm_features = rfm[['R', 'F', 'M']]

kmeans = KMeans(n_clusters=4, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_features)

# -------------------------------------------
# Visualizing the clusters
# -------------------------------------------
fig = px.scatter_3d(
    rfm,
    x='Recency',
    y='Frequency',
    z='Monetary',
    color='Cluster',
    title='Customer Segments (RFM + K-Means)',
    labels={'Recency': 'Recency (days)', 'Frequency': 'Frequency (# Orders)', 'Monetary': 'Monetary ($)'}
)
fig.show()

# From performed RFM analysis I chose segmentation for four groups. They can be interpret as the following customer segments:

Group 0: Customers who were shopping for a long time, buy moderately often and they shop for high amounts.
Group 1: Customers who have been shopping recently, they buy often and make purchases for large amounts.
Group 2: Customers who have not been buying anything for a long time, they rarely buy and make purchases for relatively small amounts.
Group 3: Customers who have been shopping recently, they rarely buy and make purchases for relatively small amounts.
Based on the received groups they can be name as follows:

Group 0 - former best customers - departing,
Group 1 - best customers - active,
Group 2 - old and random customers - inactive,
Group 3 - new and random customers - new.
Marketing strategies for the customer segments

Based on the 4 clusters may could formulate marketing strategies relevant to each group:

For the groups 1 and 3 one can apply a typical promotional strategy would focus certain efforts for the high value customers. For the best customers example actions may it be: special offers with discounts, customer cards entitling to benefits (e.g. collecting points redeemed for rewards) or a gift for the next order.

The new clients have not bought from the store before, that's why is worth fighting to transform them into best clients. Example actions for them are: e-mail marketing presenting the advantages of our products, special offers, with discounts for appropriately large purchases.

For the groups 0 and 2 i.e. former best customers and old clients also can adopt certain marketing strategy. In group 0 (former best clients) include a big customers who for some reason have stopped buying our products, then it is worth trying to renew relations with them hrough an conduct marketing campaign. The group 2 is an old and random customers and here is worth to considering whether it can to carry out some marketing activities.